In [92]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
import torch
import pickle
import glob
import os.path
import pandas as pd

import qgrid
import core.train_engine


def get_keychain_value_iter(d, key_chain=None):
    key_chain = [] if key_chain is None else list(key_chain).copy()       
    
    if not isinstance(d, dict):
        
        yield tuple(key_chain), d
    else:
        for k, v in d.items():
            yield from get_keychain_value_iter(v, key_chain + [k])
            
def get_keychain_value(d, key_chain):
    
    try:
        for k in key_chain:
            d = d[k]
            
    except Exception as ex:
        raise KeyError() from ex
        
    return d

In [94]:
training_cfg = {
    'lr': float, 
    'lr_drop_fact': float, 
    'num_epochs': int,
    'epoch_step': int,
    'batch_size': int,
    'weight_decay': float,
    'validation_ratio': float, 
}

model_cfg_meta = {
    'model_type': 'PershomModel',
    'model_kwargs': {
        'use_sup_lvlset_filt': bool,
        'filtration_kwargs': {
            'use_node_deg': bool,
            'use_node_lab': bool,
            'num_gin': int,
            'hidden_dim': int, 
            'use_mlp': bool
        }, 
        'classifier_kwargs': {
            'num_struct_elem': int
        }
    }
}

exp_cfg_meta = {
    'dataset_name': str, 
    'training': training_cfg, 
    'model': model_cfg_meta
}

exp_res_meta = {
    'exp_cfg': exp_cfg_meta, 
    'cv_test_acc': list, 
    'cv_val_acc': list, 
    'cv_indices_trn_tst_val': list, 
    'cv_epoch_loss': list,
    'start_time': list, 
    'id': str    
}

In [95]:
kc = {k: k[-1] for k, v in list(get_keychain_value_iter(core.train_engine.__exp_res_meta))}
kc

{('exp_cfg', 'dataset_name'): 'dataset_name',
 ('exp_cfg', 'training', 'lr'): 'lr',
 ('exp_cfg', 'training', 'lr_drop_fact'): 'lr_drop_fact',
 ('exp_cfg', 'training', 'num_epochs'): 'num_epochs',
 ('exp_cfg', 'training', 'epoch_step'): 'epoch_step',
 ('exp_cfg', 'training', 'batch_size'): 'batch_size',
 ('exp_cfg', 'training', 'weight_decay'): 'weight_decay',
 ('exp_cfg', 'training', 'validation_ratio'): 'validation_ratio',
 ('exp_cfg', 'model', 'model_type'): 'model_type',
 ('exp_cfg',
  'model',
  'use_super_level_set_filtration'): 'use_super_level_set_filtration',
 ('exp_cfg', 'model', 'use_node_degree'): 'use_node_degree',
 ('exp_cfg', 'model', 'use_node_label'): 'use_node_label',
 ('exp_cfg', 'model', 'gin_number'): 'gin_number',
 ('exp_cfg', 'model', 'gin_dimension'): 'gin_dimension',
 ('exp_cfg', 'model', 'gin_mlp_type'): 'gin_mlp_type',
 ('exp_cfg', 'model', 'num_struct_elements'): 'num_struct_elements',
 ('cv_test_acc',): 'cv_test_acc',
 ('cv_val_acc',): 'cv_val_acc',
 ('cv_in

In [96]:
COL_NAMES = {
    ('exp_cfg', 'dataset_name'): 'dataset_name',
#     ('exp_cfg', 'training', 'lr'): 'lr',
#     ('exp_cfg', 'training', 'lr_drop_fact'): 'lr_drop_fact',
#     ('exp_cfg', 'training', 'num_epochs'): 'num_epochs',
#     ('exp_cfg', 'training', 'epoch_step'): 'epoch_step',
#     ('exp_cfg', 'training', 'batch_size'): 'batch_size',
#     ('exp_cfg', 'training', 'weight_decay'): 'weight_decay',
#     ('exp_cfg', 'training', 'validation_ratio'): 'validation_ratio',
    ('exp_cfg', 'model', 'model_type'): 'model_type',
    ('exp_cfg', 'model', 'use_super_level_set_filtration'): 'use_super_level_set_filtration',
    ('exp_cfg', 'model', 'use_node_degree'): 'use_node_degree',
    ('exp_cfg', 'model', 'use_node_label'): 'use_node_label',
    ('exp_cfg', 'model', 'gin_number'): 'gin_number',
    ('exp_cfg', 'model', 'gin_dimension'): 'gin_dimension',
    ('exp_cfg', 'model', 'gin_mlp_type'): 'gin_mlp_type',
    ('exp_cfg', 'model', 'num_struct_elements'): 'num_struct_elements',
#     ('cv_test_acc',): 'cv_test_acc',
#     ('cv_val_acc',): 'cv_val_acc',
#     ('cv_indices_trn_tst_val',): 'cv_indices_trn_tst_val',
#     ('cv_epoch_loss',): 'cv_epoch_loss',
#     ('start_time',): 'start_time',
#     ('id',): 'id',
    ('finished_training',): 'finished_training'
}

In [97]:
def read_files(path):
    files = glob.glob(os.path.join(path, "*.pickle"))
    res = []
    for f in files: 
        res.append(pickle.load(open(f, 'rb')))

    print("# Found", len(res), "files.")
    return res


def pd_frame(path):
    
    f = read_files(path)
    
    data_frames = []
    for i, res in enumerate(f):
        row = {}
            
        cv_acc_last = [x[-1] for x in res['cv_test_acc'] if len(x) > 0]
        
        row['acc_last_mean'] = np.mean(cv_acc_last)
        row['acc_last_std'] = np.std(cv_acc_last)
        
        cv_acc_validated = []
        for test, val in zip(res['cv_test_acc'], res['cv_val_acc']):
            if not len(test) == res['exp_cfg']['training']['num_epochs']:
                continue
            n = len(test)//2
            test = torch.tensor(test[n:])
            val = torch.tensor(val[n:])
            
            _, i_max = val.max(0)
            cv_acc_validated.append(test[i_max].item())
            
        row['acc_val_mean'] = np.mean(cv_acc_validated)
        row['acc_val_std'] = np.std(cv_acc_validated)
        
        
        cv_folds_available = sum([1 for cv in res['cv_test_acc'] if len(cv) == res['exp_cfg']['training']['num_epochs']])
        row['cv_folds_available'] = cv_folds_available
        
        
        for k, v in COL_NAMES.items():
            try:
                row[v] = get_keychain_value(res, k)
            except KeyError:
                pass

        f = pd.DataFrame(row, index=[i])
        
        data_frames.append(f)
        
        
    return pd.concat(data_frames, sort=True)

In [100]:
path = './results/'
RES = pd_frame(path)
qgrid_widget = qgrid.show_grid(RES, show_toolbar=True)


# Found 21 files.


In [101]:
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:
qgrid_widget.get_changed_df()

The following cells contain some utility for messing around with results, i.e., deleting etc. 

In [19]:
# files = glob.glob(os.path.join(path, "*.pickle"))
# for f in files:     
#     with open(f, 'rb') as fid:
#         res = pickle.load(fid)
#     finished_training = (len(res['cv_test_acc'][-1]) == res['exp_cfg']['training']['num_epochs'])
#     res['finished_training'] = finished_training
    
#     with open(f, 'wb') as fid:
#         pickle.dump(obj=res, file=fid)